In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


### Настройка Spark Session

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

# Лаба 1. Расчет рейтингов фильмов – RDD

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

Задание необходимо сделать, используя RDD.


**film id for calculations** - `328`

---
## u.data

The full **u.data** set, 100000 ratings by 943 users on 1682 items.
Each user has rated at least 20 movies.  Users and items are numbered consecutively from 1.  The data is randomly ordered. This is a tab separated list of:

`user id | item id | rating | timestamp` 

*The time stamps are unix seconds since 1/1/1970 UTC*

In [4]:
rdd = sc.textFile('/labs/laba01/ml-100k/u.data').map(lambda x: x.split('\t'))

In [5]:
hist_film = (rdd
             .filter(lambda x: x[1] == '328')
             .map(lambda x: (x[2], 1))
             .reduceByKey(lambda x, y: x + y)
             .sortByKey(True, 1)
             .map(lambda x: x[1])
             .collect()
          )

hist_all = (rdd
             .map(lambda x: (x[2], 1))
             .reduceByKey(lambda x, y: x + y)
             .sortByKey(True, 1)
             .map(lambda x: x[1])
             .collect()
          )

answers = {"hist_film" : hist_film, "hist_all" : hist_all}

with open("lab01.json", "w") as f:
    json.dump(answers, f)

In [6]:
answers

{'hist_film': [12, 40, 94, 109, 40],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

## u.item     
Information about the items (movies); this is a tab separated list of:

`movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western`
          
The last 19 fields are the genres, a 1 indicates the movie is of that genre, a 0 indicates it is not; movies can be in several genres at once. The movie ids are the ones used in the **u.data** set.

In [7]:
rdd_film = sc.textFile('/labs/laba01/ml-100k/u.item').map(lambda x: x.split('|'))

In [8]:
rdd_single_film = rdd_film.filter(lambda x: x[0] == '328')

In [9]:
rdd_single_film.collect()

[['328',
  'Conspiracy Theory (1997)',
  '08-Aug-1997',
  '',
  'http://us.imdb.com/M/title-exact?Conspiracy+Theory+(1997)',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0',
  '1',
  '0',
  '0']]

### stop Spark Context

In [10]:
spark.stop()